In [1]:
import numpy as np                # Librería de álgebra lineal
import matplotlib.pyplot as plt   # Librería de gráficos
import mef

# Problema 2

Considere el entramado mostrado en la figura, con una fuerza aplicada de $20 kN$. Calcule los desplazamientos de cada uno de los nodos y las tensiones que sufre cada elemento. Todos los elementos tienen $E = 210 GPa$ y una sección de $10 cm^2$ , excepto el elemento 3, que tiene una sección de $20 cm^2$. Los elementos 2 y 5 tienen una longitud de 8 metros y el elemento 3 de  4 metros.

<center><img src="Prob 2.png"></center>


### Datos

In [2]:
#Matriz de nodos
MN = np.array([[0, 0], 
               [8, 0],
               [16, 0],
               [8, 4]])

n_nodos = MN.shape[0]

In [3]:
#Matriz de conectividad
MC = np.array([[0, 3], 
               [0, 1],
               [1, 3],
               [2, 3],
               [1, 2]])

n_element = MC.shape[0]

In [4]:
#grados de libertad
glxn = 2

In [5]:
E = 2.1E11*np.ones(5) #Modulo elastico
A = np.array([10, 10, 20, 10, 10])*1E-4 #Área

In [6]:
#Condiciones de vinculo
S = np.array([0, 1, 5])
US = np.array([0, 0, 0]).reshape([-1, 1])

R = np.array([2, 3, 4, 6, 7])
FR = np.array([0, -20000, 0, 0, 0]).reshape([-1, 1])

### Resolución

In [7]:
K = mef.Kglobal(MN, MC, E, A, glxn)

In [8]:
F, U = mef.solve(K, R, FR, S, US)

In [9]:
for nodo in range(n_nodos):
    print('\033[1m \033[12m    Nodo %s \033[0m   Ux = %2.3f mm     Uy = %6.3f mm     Fx = %3.1f kN     Fy = %5.1f kN'
          %(nodo+1, U[2*nodo]*1000, U[2*nodo+1]*1000, F[2*nodo]/1000, F[2*nodo+1]/1000))


     Nodo 1    Ux = 0.000 mm     Uy =  0.000 mm     Fx = 0.0 kN     Fy =  10.0 kN
     Nodo 2    Ux = 0.762 mm     Uy = -3.844 mm     Fx = 0.0 kN     Fy = -20.0 kN
     Nodo 3    Ux = 1.524 mm     Uy =  0.000 mm     Fx = 0.0 kN     Fy =  10.0 kN
     Nodo 4    Ux = 0.762 mm     Uy = -3.653 mm     Fx = 0.0 kN     Fy =   0.0 kN


### Cálculo de las tensiones

In [10]:
#Defino una matriz que me indique la posición final de los nodos, a partir de MN y U
MN_f = U.reshape(n_nodos, 2)+MN

In [11]:
#Calculo la lonitud inicial y final de cada elemento
Lo = np.zeros([n_element, 1])
Lf = np.zeros([n_element, 1])

for i in range(n_element):
    n1 = MC[i, 0]
    n2 = MC[i, 1]
    Lo[i] = np.sqrt((MN[n1, 0]-MN[n2, 0])**2+(MN[n1, 1]-MN[n2, 1])**2)
    Lf[i] = np.sqrt((MN_f[n1, 0]-MN_f[n2, 0])**2+(MN_f[n1, 1]-MN_f[n2, 1])**2)

In [12]:
#Calculo el delta_L de cada elemento
delta_L = Lf-Lo

In [13]:
#Calculo la deforamción y la tensión de cada elemento
epsi = np.zeros([n_element,1])
sigma = np.zeros([n_element,1])
for i in range(n_element):
    epsi[i] = delta_L[i]/Lo[i]
    sigma[i] = E[i]*epsi[i]

In [14]:
for i in range(n_element):
    print('\033[1m \033[12m    Elemento %s \033[0m   Sigma = %7.3f MPa'%(i+1, sigma[i]/1E6))

     Elemento 1    Sigma = -22.344 MPa
     Elemento 2    Sigma =  20.024 MPa
     Elemento 3    Sigma =  10.000 MPa
     Elemento 4    Sigma = -22.344 MPa
     Elemento 5    Sigma =  20.024 MPa
